In [1]:
def graph(stt,emo,filepath,savename,date):
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import wave
    from matplotlib.lines import Line2D
    from matplotlib import font_manager, rc
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
    %matplotlib inline
    stt = pd.DataFrame(stt)
    emo = pd.DataFrame(emo)
    
    
    emo2 = emo.copy()
    startcol = emo2.columns.values[(emo2==0).iloc[0,:]][0] #시작점 컬럼
    strcol = emo2.dtypes.index[emo2.dtypes == object].values[0] # 감정컬럼
    emo3 = pd.concat([emo2.drop([strcol,startcol],axis=1),emo2.iloc[:,startcol],emo2.iloc[:,strcol]],axis=1)
    emo3['color'] = emo3.iloc[:,2].copy()
    
    emo3['color'] = emo3['color'].replace(['excited','neutral','happy','sad','frustration'], ['#c0392b','#7f8c8d','#e67e22','#2980b9','#2c3e50'])
    n = 0
    sen = ""
    emo3['sen'] = np.nan
    i=0
    n=0
    for i in range(len(stt)):
        if(stt.iloc[i,1]<=emo3.iloc[n,0]):
            sen = sen+" "+stt.iloc[i,0]
        else :
            emo3['sen'].iloc[n] = sen
            n = n+1
            sen = ""
            if(n == len(emo3)):
                break
    
            
    result = emo3.dropna(axis=0)
    spf = wave.open(filepath,'r')
     
    #Extract Raw Audio from Wav File
    signal = spf.readframes(-1)
    signal = np.fromstring(signal, dtype=np.int16)
    framerate = spf.getframerate()
        
    Time = np.linspace(0,len(signal)/framerate, num=len(signal))
    
    plt.figure(figsize=(30, 8))
    ax = plt.gca()
    ax.set_facecolor('w')
    plt.title('음성 감정 분석 차트',fontdict = {'fontsize' : 50})
    for i in range(len(result)):
        D = (Time>result.iloc[i,1])&(Time<=result.iloc[i,0])
        plt.plot(Time[D],signal[D],emo3['color'].iloc[i])
        text = result.loc[i,'sen']
        
        
        if ((len(text)>80)&(len(text)<100)):
            text = text[:20]+'\n'+text[20:40]+'\n'+text[40:60]+'\n'+text[60:80]+'\n'+text[80:100]
        
        if ((len(text)>60)&(len(text)<80)):
            text = text[:20]+'\n'+text[20:40]+'\n'+text[40:60]+'\n'+text[60:80] 
        
        if ((len(text)>40)&(len(text)<60)):
            text = text[:20]+'\n'+text[20:40]+'\n'+text[40:] 
        
        if ((len(text)>20)&(len(text)<40)):
            text = text[:20]+'\n'+text[20:40] 
        
        
        plt.text(np.mean(Time[D]),max(signal)/2+max(signal[D])*0.3,text,color=result.loc[i,'color'],fontdict={'fontsize':12})
    
    
    custom_lines = [Line2D([0], [0], color='#c0392b', lw=2),
                    Line2D([0], [0], color='#7f8c8d', lw=2),
                    Line2D([0], [0], color='#e67e22', lw=2),
                    Line2D([0], [0], color='#2980b9', lw=2),
                    Line2D([0], [0], color='#2c3e50', lw=2)]
    ax.legend(custom_lines, ['excited','neutral','happy','sad','frustration'],loc='lower right', prop={'size': 12})
    
    
    plt.savefig('result/'+savename+date+'/'+savename+date+'emo.png')
    print('emograph saved in '+'result/'+savename+date+'/'+savename+date+'emo.png')